# Pandas vs PySpark — Common Functions & Mappings (Cheat Sheet)

This notebook collects common operations in **pandas** and **PySpark (DataFrame API)**, grouped by task.

- All **explanations** are in Markdown cells.
- **Examples** are in code cells.

> Note: PySpark examples require a Spark environment. If you are not running Spark locally, treat those cells as reference only.

In [4]:
# Optional setup (uncomment if you want to run cells locally)
!pip install pandas pyspark

import pandas as pd
from pyspark.sql import SparkSession, functions as F
spark = SparkSession.builder.appName('cheatsheet').getOrCreate()

## Creation & inspection

- **Create DataFrame**  
  - **pandas:** `pd.DataFrame(data)`, `pd.Series(...)`  
  - **PySpark:** `spark.createDataFrame(data)`, `spark.read.csv/json/parquet(...)`

- **Schema / info**  
  - **pandas:** `df.info()`, `df.dtypes`, `df.shape`, `df.head()`  
  - **PySpark:** `df.printSchema()`, `df.dtypes`, `df.count()`, `df.show()`


## Column selection & projection

- **Select columns**  
  - **pandas:** `df[['col1','col2']]`, `df.filter(items=[...])`  
  - **PySpark:** `df.select('col1','col2')`

- **Rename / cast**  
  - **pandas:** `df.rename(columns={'a':'b'})`, `df.astype({'a':'int32'})`  
  - **PySpark:** `df.withColumnRenamed('a','b')`, `df.withColumn('a', F.col('a').cast('int'))`

- **Add/derive column**  
  - **pandas:** `df.assign(new=df.a+df.b)` or `df['new']=...`  
  - **PySpark:** `df.withColumn('new', F.col('a') + F.col('b'))`


## Row filtering

- **Filter / where / query**  
  - **pandas:** `df[df.a > 0]`, `df.query("a > 0 and b == 'x'")`  
  - **PySpark:** `df.filter(F.col('a') > 0)`, `df.where("a > 0 AND b = 'x'")`


## Missing values

- **Detect / fill / drop**  
  - **pandas:** `df.isna()`, `df.fillna({...})`, `df.dropna(subset=[...])`  
  - **PySpark:** `df.na.fill({...})`, `df.na.drop(subset=[...])`, `F.isnan`, `F.isnull`


## Replace values

- **Replace values or with conditions**  
  - **pandas:** `df.replace({'old':'new'})`, `df.where(cond, other)`  
  - **PySpark:** `df.replace({'old':'new'})`, `F.when(cond, x).otherwise(y)`


## Sorting & deduplication

- **Sort values / index**  
  - **pandas:** `df.sort_values(['a','b'], ascending=[True, False])`  
  - **PySpark:** `df.orderBy(F.col('a').asc(), F.col('b').desc())`

- **Distinct / drop duplicates**  
  - **pandas:** `df.drop_duplicates(['key'])`  
  - **PySpark:** `df.dropDuplicates(['key'])`, `df.select('key').distinct()`


## Aggregation & GroupBy

- **Group and aggregate**  
  - **pandas:** `df.groupby('k')['v'].agg(['sum','mean'])`  
  - **PySpark:** `df.groupBy('k').agg(F.sum('v').alias('sum'), F.avg('v').alias('mean'))`

- **Counts & distinct counts**  
  - **pandas:** `df['k'].value_counts()`, `df.groupby('k')['u'].nunique()`  
  - **PySpark:** `df.groupBy('k').count()`, `df.groupBy('k').agg(F.countDistinct('u'))`

- **Descriptive stats**  
  - **pandas:** `df.describe()`, `df.quantile([0.5, 0.9])`  
  - **PySpark:** `df.describe().show()`, `df.approxQuantile('col',[0.5,0.9], 0.01)`


## Joins & unions

- **SQL-style join**  
  - **pandas:** `df1.merge(df2, on='id', how='left')`, `df1.join(df2, on='id')`  
  - **PySpark:** `df1.join(df2, on='id', how='left')`

- **Union / concat**  
  - **pandas:** `pd.concat([df1, df2], axis=0, ignore_index=True)`  
  - **PySpark:** `df1.unionByName(df2, allowMissingColumns=True)`


## Window functions (per-group running calcs)

- **Window spec & functions**  
  - **pandas:** `df.groupby('g')['v'].cumsum()`, `transform('rank')`  
  - **PySpark:**
    ```python
    from pyspark.sql.window import Window
    w = Window.partitionBy('g').orderBy('ts')
    df.withColumn('run_sum', F.sum('v').over(w)) \
      .withColumn('rank', F.rank().over(w))
    ```


## Reshaping

- **Pivot / unpivot**  
  - **pandas:** `df.pivot(index='i', columns='k', values='v')`, `pd.melt(...)`  
  - **PySpark:** `df.groupBy('i').pivot('k').agg(F.first('v'))` (no built-in melt; use selects/stack/explode patterns)


## String operations

- **Vectorized string ops**  
  - **pandas:** `df['s'].str.lower()`, `.str.contains(r'...')`, `.str.extract(r'(...)')`  
  - **PySpark:** `F.lower('s')`, `F.regexp_extract('s', r'(...)', 1)`, `F.regexp_replace('s', r'..','')`, `F.split`, `F.concat_ws`


## Dates & times

- **Parse & accessors**  
  - **pandas:** `pd.to_datetime(df['ts'])`, `df['ts'].dt.year`, `df['ts'] + pd.Timedelta(days=7)`  
  - **PySpark:** `F.to_timestamp('ts')`, `F.year('ts')`, `F.date_add('ts', 7)`, `F.months_between(...)`


## Math & conditionals

- **Column math & conditionals**  
  - **pandas:** `df['z'] = np.where(df.a>0, df.b, 0)`, `df.eval('z=a+b')`  
  - **PySpark:** `df.withColumn('z', F.when(F.col('a')>0, F.col('b')).otherwise(0))`, expressions with `F.col(...) + F.col(...)`


## Explode & arrays/structs

- **Explode arrays / build structs**  
  - **pandas:** `df.explode('arr')`  
  - **PySpark:** `F.explode('arr')`, `F.array('a','b')`, `F.struct('a','b')`


## I/O (read/write)

- **CSV / JSON / Parquet**  
  - **pandas:** `pd.read_csv(...)`, `df.to_csv(...)`; `pd.read_json(...)`; `pd.read_parquet(...)`  
  - **PySpark:** `spark.read.csv(..., header=True, inferSchema=True)`, `df.write.mode('overwrite').csv(...)`; `spark.read.json(...)`; `spark.read.parquet(...)`, `df.write.parquet(...)`


## User-defined functions (use sparingly)

- **Custom Python**  
  - **pandas:** `df.apply(func, axis=1)` (row-wise), `df['col'].map(func)`  
  - **PySpark:** `F.udf(py_func, returnType)` / `pandas_udf` for vectorized UDFs (prefer built-ins for speed)


## Tiny side-by-side examples

### Example 1 — Filter, derive, aggregate (pandas)

In [1]:
import pandas as pd

df = pd.DataFrame({'g':['A','A','B'],'a':[1,3,2],'b':[10,20,30]})
out = (
  df[df.a>1]
  .assign(z=lambda d: d.a + d.b)
  .groupby('g')['z'].agg(['sum','mean'])
)
print(out)


   sum  mean
g           
A   23  23.0
B   32  32.0


###Example 1 — Filter, derive, aggregate (pyspark)

In [ ]:
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.getOrCreate()

# Same data as your pandas example
data = [("A", 1, 10), ("A", 3, 20), ("B", 2, 30)]
df = spark.createDataFrame(data, schema=["g", "a", "b"])

out = (
    df.filter(F.col("a") > 1)              # df[df.a > 1]
      .withColumn("z", F.col("a") + F.col("b"))   # .assign(z=a+b)
      .groupBy("g")                         # .groupby('g')
      .agg(
          F.sum("z").alias("sum"),
          F.mean("z").alias("mean")
      )                                     # ['z'].agg(['sum','mean'])
)

out.show()


### Example 1 — Filter, derive, aggregate (PySpark)

In [5]:
# Requires a running Spark session named `spark`
from pyspark.sql import functions as F

df = spark.createDataFrame([('A',1,10),('A',3,20),('B',2,30)], ['g','a','b'])
out = (
  df.filter(F.col('a') > 1)
    .withColumn('z', F.col('a') + F.col('b'))
    .groupBy('g')
    .agg(F.sum('z').alias('sum'), F.avg('z').alias('mean'))
)
out.show()


+---+---+----+
|  g|sum|mean|
+---+---+----+
|  B| 32|32.0|
|  A| 23|23.0|
+---+---+----+



### Example 2 — Join + window rank (pandas)

In [6]:
import pandas as pd

left  = pd.DataFrame({'id':[1,2,3], 'grp':['A','A','B'], 'score':[5,3,8]})
right = pd.DataFrame({'id':[1,2,3], 'info':['x','y','z']})
m = left.merge(right, on='id', how='left')
m['rnk'] = m.groupby('grp')['score'].rank(ascending=False, method='first')
print(m.sort_values(['grp','rnk']))


   id grp  score info  rnk
0   1   A      5    x  1.0
1   2   A      3    y  2.0
2   3   B      8    z  1.0


### Example 2 — Join + window rank (PySpark)

In [9]:
# Requires a running Spark session named `spark`
from pyspark.sql import functions as F
from pyspark.sql.window import Window

left  = spark.createDataFrame([(1,'A',5),(2,'A',3),(3,'B',8)], ['id','grp','score'])
right = spark.createDataFrame([(1,'x'),(2,'y'),(3,'z')], ['id','info'])
m = left.join(right, on='id', how='left')
w = Window.partitionBy('grp').orderBy(F.col('score').desc())
m = m.withColumn('rnk', F.rank().over(w))
m.orderBy('grp','rnk').show()

+---+---+-----+----+---+
| id|grp|score|info|rnk|
+---+---+-----+----+---+
|  1|  A|    5|   x|  1|
|  2|  A|    3|   y|  2|
|  3|  B|    8|   z|  1|
+---+---+-----+----+---+



## Key differences to remember

- **Execution model:** pandas is *eager* (in-memory); PySpark is *lazy* & distributed (actions like `.count()`, `.show()` trigger execution).

- **Types:** pandas has rich dtypes (nullable Int64, `category`); PySpark uses Spark SQL types—casting matters.

- **Nulls:** pandas uses `NaN/NaT`; PySpark uses SQL `null`. Use `F.coalesce`, `F.when`, `na.fill` for null-safe logic.

- **Apply/UDFs:** Great in pandas; in PySpark prefer built-ins/window functions. UDFs can be slower; use `pandas_udf` if needed.

- **Performance:** Push filters/projections early; avoid wide shuffles; cache materialized results in Spark only when they are reused.
